# Assignment 1: Chinese character "detection"
LT2326, Autumn 2021

Name: Max Boholm (gusbohom)

# Introduction

This notebook cotains the code for *Assignment 1: Chinese character "detection"* for the course *Machine learning for statistical NLP: Advanced* (course code LT2326), Autumn 2021. The notebook is organized into the folowing parts:

*    Meta variables (the term *hyperparameter* is here reserved for decisions on the models), which define ... the loacation (path) of the data, ... 
*    Data preparation
*    Definition and training of two models
*    Testing and evaluation
*    ...




# TO DO

**THINK AGAIN ABOUT WHAT THE PROBLEM IS**
How do you go from the "topographic" representation back to coordinates?
Consider using coordinates as targets, how do you calculate the loss? (Perhaps not that hard... one output for every dimension of coordinates... but then: ) How to handle variable length of boxes in image?

???
predict coordinates
for a 2048 x 2048 vector translate those coordinates to 0s and 1s
compute loss for that vector



*    adjusted box vs polygon proper - how to implement the latter? Cross-secting pixels? Too complicated?!
*    Rescaling - how stupid is my approach?
*    How to build (convolutional) models?
    *    Does I have to consider padding? (If so, perhaps best to use a `torch` dataloader)
    *    Flattening
*    How to visualize?
    *    Reverse flattening
    *    Heatmap simple
    *    Heatmap-layer
*    Evaluation metrics

# Libraries

In [1]:
import glob
import json
import random
import torch
from PIL import Image
import numpy as np

# Meta variables

In [11]:
#path = "../../scratch/lt2326-h21/a1/"
path = "../develop_util/"
images_dir = path + "images/"      # the directory of images to be data
meta_ctw   = path + "info.json"   # the (path to) the general meta file of CTW 
meta_train = path + "train.jsonl" # the (path to) the file containing the annotations of CTW training data

train_proportion = 0.7 # the proportion of training data; proportion of test data will be the complement of this number

#device = torch.device('cuda:0')
device = torch.device('cpu')

restriction = None # set to an integer, if subsample of the data is to be used in e.g. the development phase
rescale_to = 256 # 




In [ ]:
def devider(n):
    numerator = 2
    while n > 1:
        n = n/numerator
        print(n, numerator)
        numerator = numerator * 2
devider(2048)

## Part 1: data preparation

In [12]:
def only_train(file_dir = images_dir, meta_file = meta_ctw):
    """Tests whether image files in a directory is part of the training set of the CTW data set, 
       as defined by the json metafile for CTW. Returns a list of filenames such that they are both
       in the specified directory and the CTW training set.
    """
    meta = json.load(open(meta_file,"r"))
    
    train_files_CTW = [entry["file_name"] for entry in meta["train"]]
    
    files_to_keep = []
    
    potential_files=[file.split("/")[-1] for file in glob.glob(file_dir+"*.jpg")]
    
    for file in potential_files:
        if file in train_files_CTW:
            files_to_keep.append(file)
            
    return files_to_keep

def CTW_mapper(files, meta = meta_train):
    """ Identifies annotations for files from the training set of the CTW dataset. 
        Returns pyhoton dictionary that maps filenames (keys) with annotations (values), 
        which like in the original format is a list of lists of json elements / python dictinaries. 
    """
    mapping = {}
   
    with open(meta, "r") as f:
        
        annotations_data = [json.loads(line) for line in f.readlines()]
        
        #print(annotations_data[1])

        for file in files:
            for annotation in annotations_data:
                if annotation["image_id"] == file[:-4]:
                    mapping[file] = annotation["annotations"]
                    break
    
    return mapping
                    
def img2vector(file, directory=images_dir, device=device, rescale=rescale_to):
    """ Takes a filename of an image in a directory and returns a ...
    """
    
    img = Image.open(directory+file)
    
    if rescale != None:
        img = img.resize((rescale_to, rescale_to))
    
    img_vec = torch.LongTensor(np.array(img), device=device) # do I really need this nesting?
    
    #flatten?
    #padding?
    
    return img_vec

def coordinates2vector(file, mapping, device=device, height = 2048, width = 2048, rescale=rescale_to): 
    """ Builds a vector of 0s and 1s based on the coordinates ...
    """
    
    #ALTHOUGH I RESCALE THE INPUT I DO NOT HAVE TO RESCALE THE OUTPUT!!!
    
    if rescale != None:
        height = rescale_to
        width = rescale_to
        rescale_by = rescale_to / height
    
    char_matrix = torch.zeros(height, width, device=device)
    
    char_areas = []
    for block in mapping[file]:
        for character in block:
            if character["is_chinese"] == True:
                xmin = int(character["adjusted_bbox"][0])
                ymin = int(character["adjusted_bbox"][1])
                w    = int(character["adjusted_bbox"][2])
                h    = int(character["adjusted_bbox"][3])
                
                if rescale != None:
                    xmin = int(xmin * rescale_by)
                    ymin = int(ymin * rescale_by)
                    w    = int(w    * rescale_by)
                    h    = int(h    * rescale_by)
                    
                
                char_areas.append((xmin, ymin, w, h))
    
    for xmin, ymin, w, h in char_areas:
        
        r1 = height - ymin - h
        r2 = r1 + h
        c1 = width - xmin
        c2 = c1 + w
        
        char_matrix[r1:r2 , c1:c2] = 1
    
    #flatten?
    
    return char_matrix
        
def data_builder(files, directory, mapping, restriction = restriction):
    """ Compiles the data set for use. Returns a list of dictionaties, specifying:
        -  the filename; key: "file"
        -  the training data (vector); key: "img_vector"
        -  the labels (a vector of 0s and 1s indicating boxes of characters in images); key: "label" 
    """
    
    data = []
    
    for file in files:
        instance = {}
        instance["file"] = file
        instance["img_vector"] = img2vector(file)
        instance["label"] = coordinates2vector(file, mapping)
        data.append(instance) 
    
    if restriction != None:
        random.shuffle(data)
        data = data[:restriction]
        
    return data
    
files = only_train()
mapping = CTW_mapper(files)
my_data = data_builder(files, images_dir, mapping)

In [ ]:
def split():
    pass

train_data, test_data = split(my_data)

In [ ]:
# do we need padding!

#to be used in training below ... 
def data_loader(data, batch):
    ...
    yield ...

### Some testing

In [ ]:
print(files)
print(mapping)
print(my_data)

In [13]:
# Testing for 1s
labels = [x["label"] for x in my_data]
i=0
j=1
for vec in labels[:3]:
    print(j)
    for x in vec:
        for y in x:
            if y != 0:
                i = i+1
    j+=1
print(i)

1
2
3
7906


### Development area

In [ ]:
import torch
t = torch.zeros(10, 10)
t[1:3, 5:7]=1 # rows, columns
print(t.shape)
t

In [ ]:
from PIL import Image
import torch
import numpy as np
img = Image.open("../develop_util/images/0000172.jpg")

img_np=np.array(img)
img_vec=torch.LongTensor(img_np)
#print(img)
img_vec

## Part 2: the models

### Model 1: ...

In [ ]:

batch_size_m1 = 1
lr_m1 = 0.01 
stride_m1 = 1
window_m1 = (1,1)
m1_name = "m1_{}{}{}".format("B", "S", "X")


#### Model definition

#### Training and saving model

In [ ]:
train = dataloader(train_data, batch=batch_size_m1)

### Model 2: ...

In [ ]:

batch_size_m2 = 1
lr_m2 = 0.01 
stride_m2 = 1
window_m2 = (1,1)
m2_name = "m2_{}{}{}".format("B", "S", "X")


#### Model definition

#### Training and saving model

In [ ]:
train = dataloader(train_data, batch=batch_size_m2)

## Part 3: testing and evaluation

### Metrics

In [ ]:
def precison
def accuracy
def recall
def mean square X

### Testing: setting hyperparameters

In [ ]:
batch
stride
iterations


### Evaluation: performance of best models

### Visualizations